<a href="https://colab.research.google.com/github/VivianOuou/NLP-Course/blob/main/course/en/chapter2/section5_Handling_multiple_sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handling multiple sequences (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [6]:
!pip install datasets evaluate transformers[sentencepiece]

成功和不成功的区别主要在于 **输入张量的维度** 是否符合模型的预期。以下是关键对比：

---

### ❌ 失败的情况（报错 `IndexError`）
```python
input_ids = torch.tensor(ids)  # 形状为 (n,)，例如 (14,)
model(input_ids)
```
- **问题**：  
  输入的 `input_ids` 是一个 **一维张量**（形状为 `(序列长度,)`），但模型默认需要 **二维张量**（形状为 `(batch_size, 序列长度)`），即必须包含批次维度。
- **错误原因**：  
  模型试图访问 `batch_size` 维度（第0维），但输入的张量只有1维（序列长度），导致维度越界。

---

### ✅ 成功的情况
```python
input_ids = torch.tensor([ids])  # 形状为 (1, n)，例如 (1, 14)
model(input_ids)
```
- **关键区别**：  
  通过 `[ids]` 将输入包装为一个列表，`torch.tensor()` 会自动将其转换为 **二维张量**，显式添加了批次维度（`batch_size=1`）。
- **符合模型预期**：  
  模型需要 `(batch_size, 序列长度)` 的输入，这里 `batch_size=1` 表示单样本批次。

---

### 🌰 直观类比
假设原始 `ids` 是 `[1, 2, 3]`：
- ❌ 错误输入：`torch.tensor([1, 2, 3])` → 形状 `(3,)`（模型无法处理）。
- ✅ 正确输入：`torch.tensor([[1, 2, 3]])` → 形状 `(1, 3)`（模型可处理）。

---

### 为什么模型需要批次输入？
1. **效率**：GPU 擅长并行计算，批量处理比逐条处理更快。
2. **统一接口**：无论输入单条还是多条数据，模型始终以批次形式处理（单条时 `batch_size=1`）。

---

### 补充：批处理多序列
如果输入多个序列，需保证长度一致（通过填充）：
```python
batched_ids = [
    [1, 2, 3],
    [1, 2, 0]  # 假设用0填充到长度3
]
input_ids = torch.tensor(batched_ids)  # 形状 (2, 3)
model(input_ids)
```

总结：**始终确保输入张量有 `batch_size` 维度**，这是成功运行的关键！

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
# This line will fail.
model(input_ids)

IndexError: too many indices for tensor of dimension 1

In [8]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

#把ids转化为tensor，再输入进行模型中
input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


### 填充输入（Padding the Inputs）

以下列表的列表无法直接转换为张量：

```python
batched_ids = [
    [200, 200, 200],
    [200, 200]
]
```

为了解决这个问题，我们可以使用 **填充（padding）** 来使张量成为矩形形状。填充通过向较短的句子添加一个特殊的 **填充标记（padding token）**，确保所有句子的长度相同。例如，如果你有 10 个句子，每个句子有 10 个单词，而 1 个句子有 20 个单词，填充会确保所有句子都变成 20 个单词。在我们的例子中，填充后的张量如下：

```python
padding_id = 100  # 假设填充标记的ID是100

batched_ids = [
    [200, 200, 200],          # 第一个句子（长度3）
    [200, 200, padding_id],   # 第二个句子（填充后长度3）
]
```

填充标记的 ID 可以通过 `tokenizer.pad_token_id` 获取。让我们使用它，并分别将两个句子单独输入模型，以及组成批次后输入模型：

```python
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]  # 句子1（长度3）
sequence2_ids = [[200, 200]]       # 句子2（长度2）
batched_ids = [
    [200, 200, 200],               # 句子1
    [200, 200, tokenizer.pad_token_id],  # 句子2（填充后长度3）
]

# 分别输入模型
print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)

# 批量输入模型
print(model(torch.tensor(batched_ids)).logits)
```

输出结果：

```
tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward>)       # 句子1单独输入
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward>)       # 句子2单独输入
tensor([[ 1.5694, -1.3895],                                 # 批量输入
        [ 1.3373, -1.2163]], grad_fn=<AddmmBackward>)
```

### 问题：批量预测的 logits 不正确
在批量预测的结果中，第二行的 logits 应该和单独输入句子2时的 logits（`[0.5803, -0.4125]`）相同，但我们却得到了完全不同的值（`[1.3373, -1.2163]`）！

### 原因：注意力机制会考虑填充标记
Transformer 模型的核心特点是 **注意力层（attention layers）**，它们会对序列中的每个 token 进行上下文建模。这意味着，填充标记（如 `padding_id=100`）也会被纳入计算，因为注意力机制会关注序列中的所有 token（包括填充 token）。

### 解决方案：使用注意力掩码（Attention Mask）
为了确保：
1. **单独输入不同长度的句子** 和  
2. **批量输入填充后的句子**  

得到相同的结果，我们需要告诉注意力层 **忽略填充标记**。这可以通过 **注意力掩码（attention mask）** 实现。

#### 注意力掩码的作用
- **1**：表示模型应关注该 token（真实 token）。  
- **0**：表示模型应忽略该 token（填充 token）。  

#### 修改后的代码（使用 `tokenizer` 自动生成掩码）
```python
# 使用 tokenizer 自动处理填充和掩码
batched_inputs = tokenizer(
    ["Sentence one", "Sentence two"],  # 两个句子
    padding=True,                      # 自动填充
    return_tensors="pt",               # 返回 PyTorch 张量
)

# 输入模型（包含 input_ids 和 attention_mask）
outputs = model(**batched_inputs)
print(outputs.logits)
```

这样，模型会正确忽略填充部分，批量计算的结果将与单独计算的结果一致。

---

### 关键总结
1. **填充（Padding）**：使所有句子长度相同，以便组成张量。  
2. **注意力掩码（Attention Mask）**：确保模型忽略填充 token，避免影响预测结果。  
3. **最佳实践**：直接使用 `tokenizer(..., padding=True, return_tensors="pt")`，让分词器自动处理填充和掩码，而非手动操作。  

> 📌 **记住**：**填充 + 掩码** 是正确处理变长序列批处理的关键！

In [10]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

In [11]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


In [13]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [14]:
#练习1
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# 句子
sequence1 = "I've been waiting for a HuggingFace course my whole life."
sequence2 = "I hate this so much!"

# 单独分词并转换为ID
tokens1 = tokenizer.tokenize(sequence1)
ids1 = tokenizer.convert_tokens_to_ids(tokens1)

tokens2 = tokenizer.tokenize(sequence2)
ids2 = tokenizer.convert_tokens_to_ids(tokens2)

print("句子1的ID:", ids1)
print("句子2的ID:", ids2)

句子1的ID: [1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]
句子2的ID: [1045, 5223, 2023, 2061, 2172, 999]


In [15]:
# 转换为张量（添加批次维度）
input_ids1 = torch.tensor([ids1])
input_ids2 = torch.tensor([ids2])

# 单独预测
output1 = model(input_ids1)
output2 = model(input_ids2)

print("句子1的logits:", output1.logits)
print("句子2的logits:", output2.logits)

句子1的logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)
句子2的logits: tensor([[ 3.1931, -2.6685]], grad_fn=<AddmmBackward0>)


In [18]:
# 计算最大长度
max_length = max(len(ids1), len(ids2))

# 对句子2填充（句子1已经是最长，无需填充）
padded_ids2 = ids2 + [tokenizer.pad_token_id] * (max_length - len(ids2))

# 填充后的批次输入
batched_ids = [
    ids1,              # 句子1（长度14）
    padded_ids2        # 句子2（填充后长度14）
]

# 创建注意力掩码（1=真实token，0=填充token）
attention_mask = [
    [1] * len(ids1) + [0] * (max_length - len(ids1)),  # 句子1（无需填充，全1）
    [1] * len(ids2) + [0] * (max_length - len(ids2))   # 句子2（前5个是真实token）
]

# 转换为张量
input_ids = torch.tensor(batched_ids)
attention_mask = torch.tensor(attention_mask)

print("填充后的input_ids:\n", input_ids)
print("注意力掩码:\n", attention_mask)

填充后的input_ids:
 tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012],
        [ 1045,  5223,  2023,  2061,  2172,   999,     0,     0,     0,     0,
             0,     0,     0,     0]])
注意力掩码:
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])


In [19]:
# 输入模型（同时传递input_ids和attention_mask）
batched_output = model(input_ids, attention_mask=attention_mask)
print("批量预测的logits:\n", batched_output.logits)

批量预测的logits:
 tensor([[-2.7276,  2.8789],
        [ 3.1931, -2.6685]], grad_fn=<AddmmBackward0>)


In [21]:
# 错误示范：不带掩码的批量输入
wrong_output = model(input_ids)  # 不传递attention_mask
print("不带掩码的logits（错误）:\n", wrong_output.logits)

不带掩码的logits（错误）:
 tensor([[-2.7276,  2.8789],
        [ 2.5423, -2.1265]], grad_fn=<AddmmBackward0>)


### 处理长序列问题（Longer Sequences）

Transformer 模型对输入序列的长度有限制（通常为 512 或 1024 个 token）。如果序列超出限制，模型会报错。以下是解决方案和示例：

---

#### ❌ 问题示例
假设有一个超长序列：
```python
long_sequence = "This is a very long sequence... [hundreds of words later] ... and it will crash the model."
```

#### ✅ 解决方案 1：截断序列（Truncation）
```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 自动截断到模型最大长度（如512）
truncated_sequence = tokenizer(
    long_sequence,
    truncation=True,  # 启用截断
    max_length=512,   # 明确指定最大长度（可选）
    return_tensors="pt"
)
print("截断后的输入长度:", len(truncated_sequence["input_ids"][0]))
```

#### ✅ 解决方案 2：使用支持长序列的模型
```python
from transformers import LongformerModel

# 加载长序列专用模型（如Longformer，支持4096 token）
model = LongformerModel.from_pretrained("allenai/longformer-base-4096")
```

---

### 关键细节
1. **默认限制**：
   - BERT类模型：通常 512 token
   - GPT类模型：通常 1024 token

2. **截断方式**：
   ```python
   # 保留开头部分（默认）
   tokenizer(sequence, truncation=True, max_length=100)

   # 保留结尾部分
   tokenizer(sequence, truncation="only_last", max_length=100)
   ```

3. **长序列模型示例**：
   - **Longformer**：支持 4096 token（适合文档处理）
   - **LED**：支持 16384 token（适合超长文本摘要）

---

### 实际应用建议
```python
# 最佳实践：同时处理填充和截断
inputs = tokenizer(
    [sequence1, sequence2],
    padding=True,      # 自动填充
    truncation=True,   # 自动截断
    max_length=512,    # 设置最大值
    return_tensors="pt"
)

outputs = model(**inputs)
```

> 📌 **注意**：如果任务必须使用完整长序列（如法律文档分析），优先选择专用模型而非截断。

In [22]:
sequence = sequence[:max_sequence_length]

NameError: name 'max_sequence_length' is not defined

这个错误是因为你直接使用了未定义的变量 `max_sequence_length`。Python会报`NameError`，表示它不知道`max_sequence_length`是什么。

### 错误原因分析
```python
sequence = sequence[:max_sequence_length]  # 报错
```
- 你试图用切片操作截取序列，但`max_sequence_length`这个变量从未被定义过
- Python不知道`max_sequence_length`应该取值多少

### 正确做法

#### 方法1：直接使用固定值
```python
max_sequence_length = 512  # 先定义这个变量
sequence = sequence[:max_sequence_length]  # 现在可以正常工作
```

#### 方法2：使用tokenizer的自动截断（推荐）
```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer(sequence, truncation=True, max_length=512)  # 自动处理截断
```

#### 方法3：获取模型的最大长度
```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
max_sequence_length = tokenizer.model_max_length  # 获取模型支持的最大长度
sequence = sequence[:max_sequence_length]
```

### 关键区别
| 方法 | 优点 | 缺点 |
|------|------|------|
| 固定值 | 简单直接 | 需要知道模型限制 |
| tokenizer自动截断 | 智能处理，保留重要部分 | 需要调用tokenizer |
| 获取模型最大长度 | 动态适应不同模型 | 仍需手动切片 |

### 最佳实践建议
```python
# 推荐做法：使用tokenizer自动处理
inputs = tokenizer(
    sequence,
    truncation=True,  # 自动截断
    max_length=512,   # 明确限制长度
    return_tensors="pt"  # 返回PyTorch张量
)
```

> 这样既避免了未定义变量的错误，又能确保正确处理序列长度限制。Tokenizer会自动处理截断位置，通常比简单的前512字符截取效果更好。